In [26]:
# import internal modules
import file_path_management as fpath
import public_library as plib
import extract_info
import parameters as params
import download_and_process_pdf as dpp

In [27]:
# import packages
import csv
import pandas as pd
import PyPDF2
import requests
import time
import os
import random
from requests.auth import HTTPProxyAuth
from bs4 import BeautifulSoup
import numpy as np
from selenium import webdriver
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.service import Service
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
import os
import re
from lxml import etree

In [28]:
# rename downloaded pdf
def rename_pdf(ind, pdf_folder, time_to_wait=60):
    newname = str(ind) + ".pdf"
    time_counter = 0
    filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa: os.path.getctime(os.path.join(pdf_folder,xa)))
    while '.part' in filename:
        # print(filename)
        # print(time_counter)
        time.sleep(1)
        time_counter += 1
        if time_counter > time_to_wait:
            raise Exception('Waited too long for file to download')
        filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa: os.path.getctime(os.path.join(pdf_folder,xa)))
    filename = max([f for f in os.listdir(pdf_folder)], key=lambda xa: os.path.getctime(os.path.join(pdf_folder,xa)))
    # filename is not a number
    if re.match(filename, "[0-9]+"):
        print('yes')
        raise Exception('File name is not a number')
    else:
        os.rename(os.path.join(pdf_folder, filename), os.path.join(pdf_folder, newname))